In [2]:
import pandas as pd
import numpy as np
import tushare as ts
import datetime
import matplotlib.pyplot as plt

In [3]:
token="818670fa68bc204c217143cdb75efeae1986031841ff8ca2c6a855bd"
ts.set_token(token)
pro = ts.pro_api()

# stock_prices = pro.index_daily(ts_code='399300.SZ', start_date='20180101', end_date='20181010')
# stock_prices['c2c'] = pd.Series(np.nan, index=stock_prices.index)
# print(stock_prices)

In [8]:
stock_code='000852.SH' #沪深300 '000300.SH'   中证500  '000905.SH'   中证1000 '000852.SH'

start_date='20231030'
end_date='20231030'
rolling_window = 60
vol_types=["close_to_close"]
# vol_types=["close_to_close", "parkinson", "garman_klass", "roger_satchell", "yang_zhang"]
display=False
whether_index=True


def volatility(vol_types,whether_index,stock_code, start_date, end_date,rolling_window=20,display=True):
    #对数据进行扩展
    new_start_date=expand_date(start_date, rolling_window)
    #获取数据
    stock_prices=get_prices(whether_index,stock_code,new_start_date,end_date, rolling_window)
    # stock_prices.to_excel('399300 data.xlsx')
    #获得开始日期
    begin_index=get_begin_index(stock_prices, start_date)
    #end_index = stock_prices['trade_date'].idxmax()
    
    #判断所需要的波动率类型
    if "close_to_close" in vol_types:
        stock_prices['close_to_close'] = close_to_close(stock_prices['close'],rolling_window)
    if "parkinson" in vol_types:
        stock_prices['parkinson'] = parkinson(stock_prices['high'],stock_prices['low'],rolling_window)
    if "garman_klass" in vol_types:
        stock_prices['garman_klass'] = garman_klass(stock_prices['open'],stock_prices['close'],stock_prices['high'],stock_prices['low'],rolling_window)
    if "roger_satchell" in vol_types:
        stock_prices['roger_satchell'] = roger_satchell(stock_prices['open'],stock_prices['close'],stock_prices['high'],stock_prices['low'],rolling_window) 
    if "yang_zhang" in vol_types:
        stock_prices['yang_zhang'] = yang_zhang(stock_prices['open'],stock_prices['close'],stock_prices['high'],stock_prices['low'],rolling_window)         
    if display==True:
        display_volatilities(stock_prices.iloc[begin_index:],vol_types)    

    volatility_data = stock_prices.iloc[begin_index:]
    
    # volatility_data.set_index('trade_date', inplace=True)

    # print(volatility_data)
    print("最新值如下")
    print(volatility_data['close_to_close'].iloc[-1])
    # print("最新值的分位数如下")
    # print(np.count_nonzero(volatility_data['close_to_close'] <= volatility_data['close_to_close'].iloc[-1]) /volatility_data['close_to_close'].size)
    # print("50%分位数如下")
    # print(np.nanpercentile(volatility_data['close_to_close'], 50))
    return volatility_data


       

def expand_date(start_date, rolling_window):
    date_time_start = datetime.datetime.strptime(start_date, '%Y%m%d')
    new_date_time = date_time_start - datetime.timedelta(days=2 * rolling_window)
    new_date_time = new_date_time.strftime('%Y%m%d')
    return new_date_time
    
def get_prices(whether_index, stock_code,start_date,end_date, rolling_window):
    if whether_index==True:
        stock_prices = pro.index_daily(ts_code=stock_code, start_date=start_date, end_date=end_date)
    elif whether_index==False:
        stock_prices = ts.pro_bar (ts_code=stock_code, adj='qfq', start_date=start_date, end_date=end_date)
        # stock_prices = pro.daily(ts_code=stock_code, start_date=start_date, end_date=end_date)  没有复权
    stock_prices.sort_values(by= 'trade_date',ascending=True, inplace = True, ignore_index=True)
    stock_prices['trade_date'] = pd.to_datetime(stock_prices['trade_date'], format="%Y%m%d")
    return stock_prices
    
def get_begin_index(stock_prices, start_date):
    begin_date = pd.Timestamp(start_date)  # 目标日期，类型为 pandas.Timestamp
    filtered_rows = stock_prices[stock_prices['trade_date'] <= begin_date]  # 筛选出日期等于或者早于目标日期的行
    begin_index = filtered_rows['trade_date'].idxmax() #因为用户输入的日期当天可能不开盘，取离不大于它且最近的开盘日期
    return begin_index


def close_to_close(close_prices, rolling_window=20):
    vol = np.sqrt(np.power(np.log(close_prices/close_prices.shift(1)), 2).rolling(window=rolling_window).mean())*np.sqrt(242)
    # vol=np.sqrt(np.log(close_prices/close_prices.shift(1)).rolling(window=rolling_window).var(ddof=1))*np.sqrt(242)
    return vol

def parkinson(highest_prices,lowest_prices,rolling_window=20):
    vol=np.sqrt((np.power(np.log(highest_prices/lowest_prices),2).rolling(window=rolling_window).mean())/np.log(16))*np.sqrt(242)
    return vol

def garman_klass(open_prices,close_prices, highest_prices,lowest_prices,rolling_window=20):
    vol=np.sqrt(np.power(np.log(highest_prices/lowest_prices),2).rolling(window=rolling_window).mean()/2-\
                (np.log(4)-1)*np.power(np.log(close_prices/open_prices), 2).rolling(window=rolling_window).mean())*np.sqrt(242)
    return vol

def roger_satchell(open_prices,close_prices, highest_prices,lowest_prices,rolling_window=20):
    vol=np.sqrt((np.log(highest_prices/close_prices)*np.log(highest_prices/open_prices)).rolling(window=rolling_window).mean()+\
                (np.log(lowest_prices/close_prices)*np.log(lowest_prices/open_prices)).rolling(window=rolling_window).mean())*np.sqrt(242)
    return vol

def yang_zhang(open_prices,close_prices, highest_prices,lowest_prices,rolling_window=20):
    k=(1.34-1)/(1.34+(rolling_window+1)/(rolling_window-1))
    vrs=(np.log(highest_prices/close_prices)*np.log(highest_prices/open_prices)).rolling(window=rolling_window).mean()+\
                (np.log(lowest_prices/close_prices)*np.log(lowest_prices/open_prices)).rolling(window=rolling_window).mean()
    v0 = np.power(np.log(open_prices/close_prices.shift(1)), 2).rolling(window=rolling_window).mean()
    vc = np.power(np.log(close_prices/open_prices), 2).rolling(window=rolling_window).mean()
    # v0= np.log(open_prices/close_prices.shift(1)).rolling(window=rolling_window).var(ddof=1)
    # vc=(np.log(close_prices/open_prices)).rolling(window=rolling_window).var(ddof=1)
    vol=(np.sqrt(v0+k*vc+(1-k)*vrs)*np.sqrt(242))
    
    return vol

def display_volatilities(stock_prices,vol_types):
    stock_price_recent=stock_prices[-30:]
    stock_price_recent.set_index('trade_date', inplace=True)
    
    # stock_columns=stock_prices[['open','close','high','low']]
    # stock_prices['open_minus_close']=stock_prices['open'].shift(-1)-stock_prices['close']
    # stock_prices=stock_prices.iloc[70:105]
    stock_columns = stock_price_recent[vol_types]
    # stock_columns=stock_prices['open_minus_close']
    stock_columns.plot()
    plt.legend()
    plt.show()
    
if __name__ == '__main__':
    volatility_data= volatility(vol_types,whether_index, stock_code, start_date, end_date,rolling_window, display)
    # volatility_data.to_excel('volatility_data.xlsx')
    print(volatility_data)

最新值如下
0.17106572505708237
      ts_code trade_date      close       open       high        low  \
79  000852.SH 2023-10-30  6006.5591  5899.3795  6017.2814  5898.6942   

    pre_close   change  pct_chg          vol        amount  close_to_close  
79  5916.6783  89.8808   1.5191  179281052.0  2.196281e+08        0.171066  


In [2]:
import numpy as np

np.arange(0, 10+1, 1)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [5]:
a = [1, 4, 3, 5]
b=[1, 2,3,4, 5]
np.union1d(a, b)

array([1, 2, 3, 4, 5])